## Script to generate NBA game predictions for 2018-2019 season

In [1]:
# Dependencies
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
import os
import bs4
import splinter
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import pymongo
import re
import time
import datetime
from datetime import date
import sqlalchemy
from sqlalchemy import create_engine

Step 1: Machine Learning Model using SVM linear classifier

In [2]:
# Read in dataset that has regular season game data from 2012-2018
df = pd.read_csv("db/2012-18_teamBoxScore_diff_columns.csv")
df.head()

,gmDate,gmTime,seasTyp,offLNm1,offFNm1,offLNm2,offFNm2,offLNm3,offFNm3,teamAbbr,...,opptFTF,diff_teamEFG%,diff_opptEFG%,diff_teamTO%,diff_opptTO%,diff_OREB%,diff_DREB%,diff_teamFTF,diff_opptFTF,outcome
0,10/30/2012,19:00,Regular,Brothers,Tony,Smith,Michael,Workman,Haywoode,WAS,...,0.278481,-0.1000,0.1000,-7.5187,7.5187,-12.8205,-12.8205,-0.056259,0.056259,0.0
1,10/30/2012,19:00,Regular,Brothers,Tony,Smith,Michael,Workman,Haywoode,CLE,...,0.222222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10/30/2012,20:00,Regular,McCutchen,Monty,Wright,Sean,Fitzgerald,Kane,BOS,...,0.405063,-0.0349,0.0349,7.5714,-7.5714,5.6006,5.6006,-0.031730,0.031730,0.0
3,10/30/2012,20:00,Regular,McCutchen,Monty,Wright,Sean,Fitzgerald,Kane,MIA,...,0.373333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10/30/2012,22:30,Regular,Foster,Scott,Zielinski,Gary,Dalen,Eric,DAL,...,0.402597,-0.0130,0.0130,-1.9419,1.9419,-10.1087,-10.1087,-0.190833,0.190833,1.0


In [3]:
df.shape

(14758, 134)

In [4]:
# drop any rows with NaN values
df = df.dropna(how='any') 

In [5]:
df.shape

(7377, 134)

In [6]:
target = df["outcome"]
target_names = ["loss", "win"]

In [7]:
# only focus on the columns for the four factors (four offense and four defense so technically 8 total)
# put that into new dataframe
df1 = df[['diff_teamEFG%','diff_opptEFG%', 'diff_teamTO%', 'diff_opptTO%', 'diff_OREB%', 'diff_DREB%', 'diff_teamFTF', 'diff_opptFTF', 'outcome']]
df1.head()

,diff_teamEFG%,diff_opptEFG%,diff_teamTO%,diff_opptTO%,diff_OREB%,diff_DREB%,diff_teamFTF,diff_opptFTF,outcome
0,-0.1000,0.1000,-7.5187,7.5187,-12.8205,-12.8205,-0.056259,0.056259,0.0
2,-0.0349,0.0349,7.5714,-7.5714,5.6006,5.6006,-0.031730,0.031730,0.0
4,-0.0130,0.0130,-1.9419,1.9419,-10.1087,-10.1087,-0.190833,0.190833,1.0
6,0.0036,-0.0036,4.6433,-4.6433,5.7300,5.7300,-0.122059,0.122059,0.0
8,0.1108,-0.1108,9.1181,-9.1181,-3.8462,-3.8462,-0.003663,0.003663,1.0


In [8]:
# new dataframe with 'outcome' dropped
data = df1.drop("outcome", axis=1)
feature_names = data.columns
data.head()

,diff_teamEFG%,diff_opptEFG%,diff_teamTO%,diff_opptTO%,diff_OREB%,diff_DREB%,diff_teamFTF,diff_opptFTF
0,-0.1000,0.1000,-7.5187,7.5187,-12.8205,-12.8205,-0.056259,0.056259
2,-0.0349,0.0349,7.5714,-7.5714,5.6006,5.6006,-0.031730,0.031730
4,-0.0130,0.0130,-1.9419,1.9419,-10.1087,-10.1087,-0.190833,0.190833
6,0.0036,-0.0036,4.6433,-4.6433,5.7300,5.7300,-0.122059,0.122059
8,0.1108,-0.1108,9.1181,-9.1181,-3.8462,-3.8462,-0.003663,0.003663


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [10]:
# Support vector machine linear classifier
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [11]:
# Model Accuracy
print('Test Acc: %.3f' % model.score(X_test, y_test))

Test Acc: 0.945


In [12]:
# Calculate classification report
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=target_names))

              precision    recall  f1-score   support

        loss       0.96      0.95      0.95      1094
         win       0.93      0.94      0.93       751

   micro avg       0.94      0.94      0.94      1845
   macro avg       0.94      0.94      0.94      1845
weighted avg       0.94      0.94      0.94      1845



In [13]:
# create function to make predictions
#
# returns string 'Win' if road team is predicted to win
# returns string 'Loss' if road team is predicted to lose
def predict_outcome_win_loss(road_team_abbr, road_team_stats, home_team_stats):
    road_team_array = np.array(road_team_stats)
    home_team_array = np.array(home_team_stats)
    diffs = road_team_array - home_team_array
    diffs_l = [diffs]
    prediction = model.predict(diffs_l)
    print(prediction)
    if ((prediction==1.).all()==True):
        print('Prediction is a Win for ' + str(road_team_abbr))
        outcome = 'Win'
    else:
        print('Prediction is a Loss for ' + str(road_team_abbr))
        outcome = 'Loss'
    return outcome

In [14]:
#Testing out prediction manually for sanity check purposes
#will automate this later on in this notebook

# Game DEN at SAC
den_stats = [.524, .513, 12.1, 12.2, 27.5, 77.7, .186, .191]
sac_stats = [.531, .536, 11.6, 13.7, 22.0, 74.8, .173, .209]
result = predict_outcome_win_loss('DEN', den_stats, sac_stats)
print(result)

[1.]
Prediction is a Win for DEN
Win


Step 2: Web scraping to capture updated daily stats from basketball-reference.com

In [15]:
#teams list
teams = ['ATL', 'BOS', 'BRK', 'CHO', 'CHI', 'CLE', 'DAL', 'DEN', 'DET',
         'GSW', 'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN',
         'NOP', 'NYK', 'OKC', 'ORL', 'PHI', 'PHO', 'POR', 'SAC', 'SAS',
         'TOR', 'UTA', 'WAS']

In [16]:
# load web pages without loading images in selenium
from selenium import webdriver

chromeOptions = webdriver.ChromeOptions()
prefs = {'profile.managed_default_content_settings.images':2}
chromeOptions.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(chrome_options=chromeOptions)

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False, options=chromeOptions)

C:\Users\zmsub\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


In [17]:
# loop to start the scraping of the stats
team_list = []
x = 0 
while x == 0:
    try:
        for team in teams:
            # URL
            url = r"https://www.basketball-reference.com/teams/" + team + "/2019.html"
            browser.visit(url)

            stat_list = []

            #Path to get titles of articles
            team_X = team
            oEfg_X = '//*[@id="team_misc"]/tbody/tr[1]/td[13]'
            dEfg_X = '//*[@id="team_misc"]/tbody/tr[1]/td[17]'
            oTOV_X = '//*[@id="team_misc"]/tbody/tr[1]/td[14]'
            dTOV_X = '//*[@id="team_misc"]/tbody/tr[1]/td[18]'
            oORB_X = '//*[@id="team_misc"]/tbody/tr[1]/td[15]'
            dDRB_X = '//*[@id="team_misc"]/tbody/tr[1]/td[19]'
            oFtByFga_X = '//*[@id="team_misc"]/tbody/tr[1]/td[16]'
            dFtByFga_X = '//*[@id="team_misc"]/tbody/tr[1]/td[20]'

            #Reading the results  
            abr = team_X
            stat_list.append(abr)
            oEfg = browser.find_by_xpath(oEfg_X).value
            stat_list.append(oEfg)
            dEfg = browser.find_by_xpath(dEfg_X).value
            stat_list.append(dEfg)
            oTOV = browser.find_by_xpath(oTOV_X).value
            stat_list.append(oTOV)
            dTOV = browser.find_by_xpath(dTOV_X).value
            stat_list.append(dTOV)
            oORB = browser.find_by_xpath(oORB_X).value
            stat_list.append(oORB)
            dDRB = browser.find_by_xpath(dDRB_X).value
            stat_list.append(dDRB)
            oFtByFga = browser.find_by_xpath(oFtByFga_X).value
            stat_list.append(oFtByFga)
            dFtByFga = browser.find_by_xpath(dFtByFga_X).value
            stat_list.append(dFtByFga)

            team_list.append(stat_list)
            print(team_list)
        x = 1
    except:
        print("try again")

[['ATL', '.511', '.545', '15.2', '13.4', '23.9', '76.7', '.193', '.234']]
[['ATL', '.511', '.545', '15.2', '13.4', '23.9', '76.7', '.193', '.234'], ['BOS', '.531', '.505', '11.5', '13.8', '22.5', '77.8', '.175', '.210']]
[['ATL', '.511', '.545', '15.2', '13.4', '23.9', '76.7', '.193', '.234'], ['BOS', '.531', '.505', '11.5', '13.8', '22.5', '77.8', '.175', '.210'], ['BRK', '.524', '.524', '12.6', '11.4', '25.1', '76.6', '.215', '.204']]
[['ATL', '.511', '.545', '15.2', '13.4', '23.9', '76.7', '.193', '.234'], ['BOS', '.531', '.505', '11.5', '13.8', '22.5', '77.8', '.175', '.210'], ['BRK', '.524', '.524', '12.6', '11.4', '25.1', '76.6', '.215', '.204'], ['CHO', '.519', '.527', '10.7', '12.3', '21.4', '76.4', '.203', '.185']]
[['ATL', '.511', '.545', '15.2', '13.4', '23.9', '76.7', '.193', '.234'], ['BOS', '.531', '.505', '11.5', '13.8', '22.5', '77.8', '.175', '.210'], ['BRK', '.524', '.524', '12.6', '11.4', '25.1', '76.6', '.215', '.204'], ['CHO', '.519', '.527', '10.7', '12.3', '21.4'

[['ATL', '.511', '.545', '15.2', '13.4', '23.9', '76.7', '.193', '.234'], ['BOS', '.531', '.505', '11.5', '13.8', '22.5', '77.8', '.175', '.210'], ['BRK', '.524', '.524', '12.6', '11.4', '25.1', '76.6', '.215', '.204'], ['CHO', '.519', '.527', '10.7', '12.3', '21.4', '76.4', '.203', '.185'], ['CHI', '.499', '.522', '13.5', '12.3', '17.0', '75.9', '.174', '.186'], ['CLE', '.494', '.555', '11.8', '11.0', '24.5', '76.3', '.182', '.179'], ['DAL', '.522', '.520', '13.6', '12.3', '23.5', '77.8', '.236', '.185'], ['DEN', '.524', '.516', '11.9', '12.2', '27.9', '77.7', '.186', '.190'], ['DET', '.495', '.520', '12.6', '12.6', '25.4', '79.8', '.199', '.232'], ['GSW', '.553', '.516', '12.6', '11.1', '22.8', '78.1', '.202', '.202'], ['HOU', '.535', '.529', '12.3', '12.4', '25.2', '75.4', '.219', '.218'], ['IND', '.535', '.504', '13.1', '14.3', '22.1', '77.2', '.178', '.176'], ['LAC', '.529', '.511', '12.5', '10.5', '22.8', '75.3', '.271', '.220'], ['LAL', '.536', '.503', '13.2', '11.8', '21.6', '7

[['ATL', '.511', '.545', '15.2', '13.4', '23.9', '76.7', '.193', '.234'], ['BOS', '.531', '.505', '11.5', '13.8', '22.5', '77.8', '.175', '.210'], ['BRK', '.524', '.524', '12.6', '11.4', '25.1', '76.6', '.215', '.204'], ['CHO', '.519', '.527', '10.7', '12.3', '21.4', '76.4', '.203', '.185'], ['CHI', '.499', '.522', '13.5', '12.3', '17.0', '75.9', '.174', '.186'], ['CLE', '.494', '.555', '11.8', '11.0', '24.5', '76.3', '.182', '.179'], ['DAL', '.522', '.520', '13.6', '12.3', '23.5', '77.8', '.236', '.185'], ['DEN', '.524', '.516', '11.9', '12.2', '27.9', '77.7', '.186', '.190'], ['DET', '.495', '.520', '12.6', '12.6', '25.4', '79.8', '.199', '.232'], ['GSW', '.553', '.516', '12.6', '11.1', '22.8', '78.1', '.202', '.202'], ['HOU', '.535', '.529', '12.3', '12.4', '25.2', '75.4', '.219', '.218'], ['IND', '.535', '.504', '13.1', '14.3', '22.1', '77.2', '.178', '.176'], ['LAC', '.529', '.511', '12.5', '10.5', '22.8', '75.3', '.271', '.220'], ['LAL', '.536', '.503', '13.2', '11.8', '21.6', '7

[['ATL', '.511', '.545', '15.2', '13.4', '23.9', '76.7', '.193', '.234'], ['BOS', '.531', '.505', '11.5', '13.8', '22.5', '77.8', '.175', '.210'], ['BRK', '.524', '.524', '12.6', '11.4', '25.1', '76.6', '.215', '.204'], ['CHO', '.519', '.527', '10.7', '12.3', '21.4', '76.4', '.203', '.185'], ['CHI', '.499', '.522', '13.5', '12.3', '17.0', '75.9', '.174', '.186'], ['CLE', '.494', '.555', '11.8', '11.0', '24.5', '76.3', '.182', '.179'], ['DAL', '.522', '.520', '13.6', '12.3', '23.5', '77.8', '.236', '.185'], ['DEN', '.524', '.516', '11.9', '12.2', '27.9', '77.7', '.186', '.190'], ['DET', '.495', '.520', '12.6', '12.6', '25.4', '79.8', '.199', '.232'], ['GSW', '.553', '.516', '12.6', '11.1', '22.8', '78.1', '.202', '.202'], ['HOU', '.535', '.529', '12.3', '12.4', '25.2', '75.4', '.219', '.218'], ['IND', '.535', '.504', '13.1', '14.3', '22.1', '77.2', '.178', '.176'], ['LAC', '.529', '.511', '12.5', '10.5', '22.8', '75.3', '.271', '.220'], ['LAL', '.536', '.503', '13.2', '11.8', '21.6', '7

In [18]:
# Create a dataframe that has updated stats for all of the teams
stats_df = pd.DataFrame(team_list)
stats_df.columns = ['Team_abbr','Offense_eFG','Defense_eFG','Offense_TOV','Defense_TOV','Offense_ORB','Defense_DRB','Offense_FtFga','Defense_FtFga']
stats_df

,Team_abbr,Offense_eFG,Defense_eFG,Offense_TOV,Defense_TOV,Offense_ORB,Defense_DRB,Offense_FtFga,Defense_FtFga
0,ATL,.511,.545,15.2,13.4,23.9,76.7,.193,.234
1,BOS,.531,.505,11.5,13.8,22.5,77.8,.175,.210
2,BRK,.524,.524,12.6,11.4,25.1,76.6,.215,.204
3,CHO,.519,.527,10.7,12.3,21.4,76.4,.203,.185
4,CHI,.499,.522,13.5,12.3,17.0,75.9,.174,.186
5,CLE,.494,.555,11.8,11.0,24.5,76.3,.182,.179
6,DAL,.522,.520,13.6,12.3,23.5,77.8,.236,.185
7,DEN,.524,.516,11.9,12.2,27.9,77.7,.186,.190
8,DET,.495,.520,12.6,12.6,25.4,79.8,.199,.232
9,GSW,.553,.516,12.6,11.1,22.8,78.1,.202,.202


Step 3: Update database (sqlite file) with 8 factors for each team (in 'stats' table)

In [19]:
# setup database connection
stats_engine = create_engine('sqlite:///db/schedule_abr.sqlite')
# send stats dataframe to stats table in database file
stats_df.to_sql('stats', stats_engine, if_exists='replace', index=False)

In [20]:
# test that we can query from stats table in db
stats_sql_df = pd.read_sql_query('SELECT * FROM stats',stats_engine)
stats_sql_df.head()

,Team_abbr,Offense_eFG,Defense_eFG,Offense_TOV,Defense_TOV,Offense_ORB,Defense_DRB,Offense_FtFga,Defense_FtFga
0,ATL,.511,.545,15.2,13.4,23.9,76.7,.193,.234
1,BOS,.531,.505,11.5,13.8,22.5,77.8,.175,.210
2,BRK,.524,.524,12.6,11.4,25.1,76.6,.215,.204
3,CHO,.519,.527,10.7,12.3,21.4,76.4,.203,.185
4,CHI,.499,.522,13.5,12.3,17.0,75.9,.174,.186


In [21]:
# NEED TO SET PRIMARY KEY after to_sql is done so that later sqlalchemy can interact with the stats table

import sqlite3
#connect to the database
conn = sqlite3.connect('db/schedule_abr.sqlite')
c = conn.cursor()

c.executescript('''
    PRAGMA foreign_keys=off;

    BEGIN TRANSACTION;
    ALTER TABLE stats RENAME TO old_stats;

    /*create a new table with the same column names and types while
    defining a primary key for the desired column*/
    CREATE TABLE stats (Team_abbr text NOT NULL PRIMARY KEY,
                            Offense_eFG REAL NOT NULL,
                            Defense_eFG REAL NOT NULL,
                            Offense_TOV REAL NOT NULL,
                            Defense_TOV REAL NOT NULL,
                            Offense_ORB REAL NOT NULL,
                            Defense_DRB REAL NOT NULL,
                            Offense_FtFga REAL NOT NULL,
                            Defense_FtFga REAL NULL);

    INSERT INTO stats SELECT * FROM old_stats;

    DROP TABLE old_stats;
    COMMIT TRANSACTION;

    PRAGMA foreign_keys=on;''')

#close out the connection
c.close()
conn.close()

Step 4: Find out each game that is being played today from database

In [22]:
# setup database connection
schedule_engine = create_engine('sqlite:///db/schedule_abr.sqlite')
# test that we can query from nba_2018_2019_schedule_logo table in database file
schedule_sql_df = pd.read_sql_query('SELECT * FROM nba_2018_2019_schedule_logo',schedule_engine)
schedule_sql_df.head()

,game_id,date,time,location,home_team,road_team,home_score,road_score,result,home_team_abr,road_team_abr,road_win_prediction,home_team_logo,road_team_logo
0,1,10/16/2018,8:00 PM,TD Garden,Boston Celtics,Philadelphia 76ers,105,87,105 - 87,BOS,PHI,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
1,2,10/16/2018,10:30 PM,ORACLE Arena,Golden State Warriors,Oklahoma City Thunder,108,100,108 - 100,GSW,OKC,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
2,3,10/17/2018,7:00 PM,Spectrum Center,Charlotte Hornets,Milwaukee Bucks,112,113,112 - 113,CHO,MIL,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
3,4,10/17/2018,7:00 PM,Little Caesars Arena,Detroit Pistons,Brooklyn Nets,103,100,103 - 100,DET,BRK,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
4,5,10/17/2018,7:00 PM,Bankers Life Fieldhouse,Indiana Pacers,Memphis Grizzlies,111,83,111 - 83,IND,MEM,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...


In [23]:
# function to get today's date in same format as 'date' column of nba_2018_2019_schedule_logo table in database
# format is mm/dd/yyyy example: 1/6/2019

def get_todays_date():
    today = str(date.today())
    todays_date = date.today().strftime('%m/%d/%Y')
    print(todays_date)
    todays_date_month_first_two = todays_date[0:2]
    print(todays_date_month_first_two)
    newstr = todays_date_month_first_two
    if todays_date_month_first_two[0] == '0':
        newstr = todays_date_month_first_two.replace("0", "")
    print(newstr)
    todays_date_day_first_two = todays_date[3:5]
    print(todays_date_day_first_two)
    newstr2 = todays_date_day_first_two
    if todays_date_day_first_two[0] == '0':
        newstr2 = todays_date_day_first_two.replace("0", "")
    print(newstr2)
    newstr3 = todays_date[5:]
    final_date_string = newstr + '/' + newstr2 + newstr3
    return final_date_string

In [24]:
# test out function to get today's date
schedule_date = get_todays_date()
schedule_date

01/06/2019
01
1
06
6


'1/6/2019'

In [25]:
# function to get only today's games from schedule table into a dataframe
# this queries database

def get_todays_games(sch_date):
    cmd1 = 'SELECT * FROM nba_2018_2019_schedule_logo WHERE date='
    print(cmd1)
    cmd2 = "'"
    print(cmd2)
    cmd3 = "'"
    print(cmd3)
    cmd = cmd1 + cmd2 + schedule_date + cmd3
    print(cmd)
    schedule_today_df = pd.read_sql_query(cmd,schedule_engine)
    return schedule_today_df

In [26]:
# get only today's games into a dataframe
todays_games_df = get_todays_games(schedule_date)
todays_games_df

SELECT * FROM nba_2018_2019_schedule_logo WHERE date=
'
'
SELECT * FROM nba_2018_2019_schedule_logo WHERE date='1/6/2019'


,game_id,date,time,location,home_team,road_team,home_score,road_score,result,home_team_abr,road_team_abr,road_win_prediction,home_team_logo,road_team_logo
0,584,1/6/2019,3:30 PM,United Center,Chicago Bulls,Brooklyn Nets,,,,CHI,BRK,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
1,585,1/6/2019,3:30 PM,Target Center,Minnesota Timberwolves,Los Angeles Lakers,,,,MIN,LAL,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
2,586,1/6/2019,3:30 PM,Staples Center,LA Clippers,Orlando Magic,,,,LAC,ORL,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
3,587,1/6/2019,6:00 PM,State Farm Arena,Atlanta Hawks,Miami Heat,,,,ATL,MIA,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
4,588,1/6/2019,7:00 PM,Chesapeake Energy Arena,Oklahoma City Thunder,Washington Wizards,,,,OKC,WAS,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
5,589,1/6/2019,7:30 PM,Scotiabank Arena,Toronto Raptors,Indiana Pacers,,,,TOR,IND,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
6,590,1/6/2019,8:00 PM,Talking Stick Resort Arena,Phoenix Suns,Charlotte Hornets,,,,PHO,CHO,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...


Step 5: 

a) get updated stats into arrays for road and home team's for each game being played today
b) run prediction for road team for each game being played today

In [27]:
# function to get updated stats for an individual team using stats dataframe

def capture_updated_stats(team_name, df):
    team_row = df.loc[df['Team_abbr'] == team_name]
    print('Capturing stats for ' + team_name)
    team_O_EFG = float(team_row.iloc[0][1])
    team_D_EFG = float(team_row.iloc[0][2])
    team_O_TOV = float(team_row.iloc[0][3])
    team_D_TOV = float(team_row.iloc[0][4])
    team_O_ORB = float(team_row.iloc[0][5])
    team_D_DRB = float(team_row.iloc[0][6])
    team_O_FTF = float(team_row.iloc[0][7])
    team_D_FTF = float(team_row.iloc[0][8])
    print('values after iloc and converting to float are...')
    print(team_O_EFG)
    print(team_D_EFG)
    print(team_O_TOV)
    print(team_D_TOV)
    print(team_O_ORB)
    print(team_D_DRB)
    print(team_O_FTF)
    print(team_D_FTF)
    team_stats_array = [team_O_EFG, team_D_EFG, team_O_TOV, team_D_TOV, team_O_ORB, team_D_DRB, team_O_FTF, team_D_FTF]
    return team_stats_array

In [28]:
# function to iterate through all rows of schedule dataframe
# capture stats for road team and home team
# prediction for road team is made ('Win' or 'Loss') and dataframe is modified

def prediction_iterrow(sch_df, sta_df):
    for (i, row) in sch_df.iterrows():
        home_team_abr_val = row['home_team_abr']
        road_team_abr_val = row['road_team_abr']
        road_win_prediction_val = row['road_win_prediction']
        print(home_team_abr_val, road_team_abr_val, road_win_prediction_val)
        road_team_stats = capture_updated_stats(road_team_abr_val,sta_df)
        print('Road team stats--------->')
        print(road_team_stats)
        home_team_stats = capture_updated_stats(home_team_abr_val,sta_df)
        print('Home team stats--------->')
        print(home_team_stats)
        our_prediction = predict_outcome_win_loss(road_team_abr_val, road_team_stats, home_team_stats)
        print(our_prediction)
        print('Our Prediction for ' + road_team_abr_val + ' vs. ' + home_team_abr_val + ' is: ' + our_prediction)
        sch_df.at[i,'road_win_prediction'] = our_prediction

In [29]:
# create a today's stats dataframe equal to stats_sql_df we had gotten from scraping earlier
todays_stats_df = stats_sql_df
todays_stats_df.head()

,Team_abbr,Offense_eFG,Defense_eFG,Offense_TOV,Defense_TOV,Offense_ORB,Defense_DRB,Offense_FtFga,Defense_FtFga
0,ATL,.511,.545,15.2,13.4,23.9,76.7,.193,.234
1,BOS,.531,.505,11.5,13.8,22.5,77.8,.175,.210
2,BRK,.524,.524,12.6,11.4,25.1,76.6,.215,.204
3,CHO,.519,.527,10.7,12.3,21.4,76.4,.203,.185
4,CHI,.499,.522,13.5,12.3,17.0,75.9,.174,.186


In [30]:
# show today's games dataframe again
todays_games_df.head()

,game_id,date,time,location,home_team,road_team,home_score,road_score,result,home_team_abr,road_team_abr,road_win_prediction,home_team_logo,road_team_logo
0,584,1/6/2019,3:30 PM,United Center,Chicago Bulls,Brooklyn Nets,,,,CHI,BRK,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
1,585,1/6/2019,3:30 PM,Target Center,Minnesota Timberwolves,Los Angeles Lakers,,,,MIN,LAL,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
2,586,1/6/2019,3:30 PM,Staples Center,LA Clippers,Orlando Magic,,,,LAC,ORL,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
3,587,1/6/2019,6:00 PM,State Farm Arena,Atlanta Hawks,Miami Heat,,,,ATL,MIA,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
4,588,1/6/2019,7:00 PM,Chesapeake Energy Arena,Oklahoma City Thunder,Washington Wizards,,,,OKC,WAS,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...


In [31]:
# run function that will use todays_stats_df to modify road_win_prediction column in todays_games_df

prediction_iterrow(todays_games_df, todays_stats_df)

CHI BRK Loss
Capturing stats for BRK
values after iloc and converting to float are...
0.524
0.524
12.6
11.4
25.1
76.6
0.215
0.204
Road team stats--------->
[0.524, 0.524, 12.6, 11.4, 25.1, 76.6, 0.215, 0.204]
Capturing stats for CHI
values after iloc and converting to float are...
0.499
0.522
13.5
12.3
17.0
75.9
0.174
0.186
Home team stats--------->
[0.499, 0.522, 13.5, 12.3, 17.0, 75.9, 0.174, 0.186]
[1.]
Prediction is a Win for BRK
Win
Our Prediction for BRK vs. CHI is: Win
MIN LAL Loss
Capturing stats for LAL
values after iloc and converting to float are...
0.536
0.503
13.2
11.8
21.6
76.4
0.178
0.184
Road team stats--------->
[0.536, 0.503, 13.2, 11.8, 21.6, 76.4, 0.178, 0.184]
Capturing stats for MIN
values after iloc and converting to float are...
0.507
0.53
11.1
13.2
24.5
74.2
0.214
0.183
Home team stats--------->
[0.507, 0.53, 11.1, 13.2, 24.5, 74.2, 0.214, 0.183]
[1.]
Prediction is a Win for LAL
Win
Our Prediction for LAL vs. MIN is: Win
LAC ORL Loss
Capturing stats for ORL
val

In [32]:
# now road_win_prediction column in todays_games_df should be updated with our predictions
todays_games_df

,game_id,date,time,location,home_team,road_team,home_score,road_score,result,home_team_abr,road_team_abr,road_win_prediction,home_team_logo,road_team_logo
0,584,1/6/2019,3:30 PM,United Center,Chicago Bulls,Brooklyn Nets,,,,CHI,BRK,Win,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
1,585,1/6/2019,3:30 PM,Target Center,Minnesota Timberwolves,Los Angeles Lakers,,,,MIN,LAL,Win,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
2,586,1/6/2019,3:30 PM,Staples Center,LA Clippers,Orlando Magic,,,,LAC,ORL,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
3,587,1/6/2019,6:00 PM,State Farm Arena,Atlanta Hawks,Miami Heat,,,,ATL,MIA,Win,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
4,588,1/6/2019,7:00 PM,Chesapeake Energy Arena,Oklahoma City Thunder,Washington Wizards,,,,OKC,WAS,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
5,589,1/6/2019,7:30 PM,Scotiabank Arena,Toronto Raptors,Indiana Pacers,,,,TOR,IND,Win,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
6,590,1/6/2019,8:00 PM,Talking Stick Resort Arena,Phoenix Suns,Charlotte Hornets,,,,PHO,CHO,Win,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...


In [33]:
# update SQL database in a table called 'today_predictions' with our predictions for today's games
schedule_abr_engine = create_engine('sqlite:///db/schedule_abr.sqlite')
todays_games_df.to_sql('today_predictions', schedule_abr_engine, if_exists='replace', index=False)



In [34]:
# test that we can query from today_predictions table in db
today_predictions_sql_df = pd.read_sql_query('SELECT * FROM today_predictions',schedule_abr_engine)
today_predictions_sql_df.head()

,game_id,date,time,location,home_team,road_team,home_score,road_score,result,home_team_abr,road_team_abr,road_win_prediction,home_team_logo,road_team_logo
0,584,1/6/2019,3:30 PM,United Center,Chicago Bulls,Brooklyn Nets,,,,CHI,BRK,Win,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
1,585,1/6/2019,3:30 PM,Target Center,Minnesota Timberwolves,Los Angeles Lakers,,,,MIN,LAL,Win,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
2,586,1/6/2019,3:30 PM,Staples Center,LA Clippers,Orlando Magic,,,,LAC,ORL,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
3,587,1/6/2019,6:00 PM,State Farm Arena,Atlanta Hawks,Miami Heat,,,,ATL,MIA,Win,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
4,588,1/6/2019,7:00 PM,Chesapeake Energy Arena,Oklahoma City Thunder,Washington Wizards,,,,OKC,WAS,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...


In [35]:
# NEED TO SET PRIMARY KEY after to_sql is done so that later sqlalchemy can interact with the today_predictions table

import sqlite3
#connect to the database
conn = sqlite3.connect('db/schedule_abr.sqlite')
c = conn.cursor()

c.executescript('''
    PRAGMA foreign_keys=off;

    BEGIN TRANSACTION;
    ALTER TABLE today_predictions RENAME TO old_table;

    /*create a new table with the same column names and types while
    defining a primary key for the desired column*/
    CREATE TABLE today_predictions (game_id INTEGER NOT NULL PRIMARY KEY,
                            date text NOT NULL,
                            time text NOT NULL,
                            location text NOT NULL,
                            home_team text NOT NULL,
                            road_team text NOT NULL,
                            home_score INTEGER NOT NULL,
                            road_score INTEGER NOT NULL,
                            result text NOT NULL,
                            home_team_abr text NOT NULL,
                            road_team_abr text NOT NULL,
                            road_win_prediction text NOT NULL,
                            home_team_logo text NOT NULL,
                            road_team_logo text NOT NULL);

    INSERT INTO today_predictions SELECT * FROM old_table;

    DROP TABLE old_table;
    COMMIT TRANSACTION;

    PRAGMA foreign_keys=on;''')

#close out the connection
c.close()
conn.close()